In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [3]:
# Load the dataset
data = pd.read_csv('Creditcard_data.csv')

In [4]:
# Replace erroneous data (if any)
data = data.replace('lakshit gupta', 'lakshit gupta')  # Example of data cleaning

In [5]:
# Split the dataset into features and target variable
y = data['Class']
x = data.drop('Class', axis=1)

In [6]:
# Perform SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
x_smote, y_smote = smote.fit_resample(x, y)

In [7]:
# Combine the resampled data into a single DataFrame
balanced_data = pd.concat([
    pd.DataFrame(x_smote, columns=x.columns),
    pd.DataFrame(y_smote, columns=['Class'])
], axis=1)

In [8]:
# Generate samples using different strategies
sample1 = balanced_data.sample(frac=0.2, random_state=42)
sample2 = balanced_data.groupby('Class', group_keys=False).apply(
    lambda grp: grp.sample(frac=0.2, random_state=42)
)
k = len(balanced_data) // int(0.2 * len(balanced_data))
start = np.random.randint(0, k)
sample3 = balanced_data.iloc[start::k]

C:\Users\user\AppData\Local\Temp\ipykernel_9648\375357354.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample2 = balanced_data.groupby('Class', group_keys=False).apply(


In [9]:
# Add clusters for sampling
num_clusters = 5
balanced_data['Cluster'] = np.arange(len(balanced_data)) % num_clusters
selected_cluster = np.random.choice(num_clusters)
sample4 = balanced_data[balanced_data['Cluster'] == selected_cluster].drop('Cluster', axis=1)

In [10]:
# Generate another sample with replacement
sample5 = balanced_data.sample(n=int(0.2 * len(balanced_data)), replace=True, random_state=42)

In [11]:
# Define models to evaluate
models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    "SVM": SVC(),
    "k-NN": KNeighborsClassifier()
}

In [12]:
# Initialize a dictionary to store results
results = {}

# List of samples to evaluate
samples = [sample1, sample2, sample3, sample4, sample5]

In [13]:
# Evaluate models on each sample
for model_name, model in models.items():
    results[model_name] = []
    for sample in samples:
        # Split the sample into training and testing sets
        X_sample = sample.drop('Class', axis=1)
        y_sample = sample['Class']
        X_train, X_test, y_train, y_test = train_test_split(
            X_sample, y_sample, test_size=0.2, random_state=42
        )

        # Train and evaluate the model
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        results[model_name].append(accuracy)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

In [14]:
# Save results to a CSV file
results_df = pd.DataFrame(results, index=["Sample1", "Sample2", "Sample3", "Sample4", "Sample5"])
results_df.to_csv("model_accuracies.csv")

In [15]:
# Print results for quick reference
print("Model accuracies saved to 'model_accuracies.csv'")

Model accuracies saved to 'model_accuracies.csv'
